In [1]:
!pip install fastjet
!pip install vector
!pip install uproot
!pip install torch
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.4/871.4 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 638.7/638.7 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.3/362.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import uproot
import torch
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing, global_mean_pool
from torch_geometric.utils import add_self_loops
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader
import itertools
from fastjet import PseudoJet, JetDefinition, ClusterSequence, antikt_algorithm, sorted_by_pt
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [3]:
from google.colab import drive
drive.mount('/content/drive')
data_file = "/content/drive/MyDrive/ML_HEP/Data_files/fcc_ee_ZH_H_ssbar_ecm_240GeV.root"
tree = uproot.open(data_file)["events"]

Mounted at /content/drive


In [4]:
tree.num_entries # total number of events

100000

In [5]:
tree.keys()

['Muon#0',
 'Muon#0/Muon#0.index',
 'Muon#0/Muon#0.collectionID',
 'ReconstructedParticles',
 'ReconstructedParticles/ReconstructedParticles.type',
 'ReconstructedParticles/ReconstructedParticles.energy',
 'ReconstructedParticles/ReconstructedParticles.momentum.x',
 'ReconstructedParticles/ReconstructedParticles.momentum.y',
 'ReconstructedParticles/ReconstructedParticles.momentum.z',
 'ReconstructedParticles/ReconstructedParticles.referencePoint.x',
 'ReconstructedParticles/ReconstructedParticles.referencePoint.y',
 'ReconstructedParticles/ReconstructedParticles.referencePoint.z',
 'ReconstructedParticles/ReconstructedParticles.charge',
 'ReconstructedParticles/ReconstructedParticles.mass',
 'ReconstructedParticles/ReconstructedParticles.goodnessOfPID',
 'ReconstructedParticles/ReconstructedParticles.covMatrix[10]',
 'ReconstructedParticles/ReconstructedParticles.clusters_begin',
 'ReconstructedParticles/ReconstructedParticles.clusters_end',
 'ReconstructedParticles/ReconstructedParti

In [6]:
tree.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
Muon#0               | vector<podio::ObjectID>  | AsGroup(<TBranchElement 'Mu...
Muon#0/Muon#0.index  | int32_t[]                | AsJagged(AsDtype('>i4'))
Muon#0/Muon#0.col... | int32_t[]                | AsJagged(AsDtype('>i4'))
ReconstructedPart... | vector<edm4hep::Recon... | AsGroup(<TBranchElement 'Re...
ReconstructedPart... | int32_t[]                | AsJagged(AsDtype('>i4'))
ReconstructedPart... | float[]                  | AsJagged(AsDtype('>f4'))
ReconstructedPart... | float[]                  | AsJagged(AsDtype('>f4'))
ReconstructedPart... | float[]                  | AsJagged(AsDtype('>f4'))
ReconstructedPart... | float[]                  | AsJagged(AsDtype('>f4'))
ReconstructedPart... | float[]                  | AsJagged(AsDtype('>f4'))
ReconstructedPart... | float[]                  | AsJagged(AsDtype('>f4'))
R

In [7]:
pid = tree['Particle.PDG'].array() # Ids of particles
p_status = tree['Particle.generatorStatus'].array() # Generator status of particles. The generator status is 1 for stable particles.
charge = tree['Particle.charge'].array() # Charges of particles
m = tree['Particle.mass'].array() # Masses of particles
px = tree['Particle.momentum.x'].array() # X-momentum of particles
py = tree['Particle.momentum.y'].array() # Y-momentum of particles
pz = tree['Particle.momentum.z'].array() # Z-momentum of particles
p_begin = tree['Particle.parents_begin'].array() # Indices of first parent
p_end = tree['Particle.parents_end'].array() # Indices of last parent
p_ind = tree['Particle#0.index'].array() # Indices of parents

In [8]:
# Function to calculate energy
def energy(m, px, py, pz):
    E = np.sqrt( m**2 + px**2 + py**2 + pz**2)
    return E

In [9]:
e = energy(m, px, py, pz) # Energy of particles

In [10]:
def get_parent_ids(i, particle_ids, parent_indices, parents_begin, parents_end) :
    """
    Find parent ID and it's index in the particle IDs list.

    Parameters:
        i (integer): Index of the stable particle.
        particle_ids (list): A list of IDs of particles.
        parent_indices (list): A list of parent indices of particles.
        parents_begin (list): A list of indices of the first parent of each particle.
        parents_end (list): A list of indices of the last parent of each particle.

    Returns:
        parent_id (integer): Parent ID of the stable particle.
        parent_indx (integer): Index of the parent particle.

    """
    pb = parents_begin[i]
    pe = parents_end[i]

    if pb == pe :
      return 0
    else :
      for id in range(pb, pe) :
        parent_indx = parent_indices[id]
      return parent_indx

In [11]:
def find_parent_and_daughter_ids(indices, particle_ids, all_particle_ids):
    """
    Find parent IDs and corresponding daughter IDs for each repeated index.

    Parameters:
        indices (list): A list of parent indices corresponding to stable particles.
        particle_ids (list): A list of IDs of stable particles.
        all_particle_ids (list): A list of all particle IDs.

    Returns:
        list: [[parent index, parent ID, list of daughter IDs],...,....].
    """
    # Count occurrences of each index
    counts = Counter(indices)

    # Identify repeated indices
    repeated_indices = {index for index, count in counts.items() if count >= 2}

    # Collect parent and daughter IDs for repeated indices
    parent_daughters = []
    for index in repeated_indices:
        parent_id = all_particle_ids[index]  # Get the parent ID using the index
        daughters = [particle_ids[i] for i in range(len(indices)) if indices[i] == index]
        parent_daughters.append([index, parent_id, daughters])

    return parent_daughters

In [13]:
def get_graph_list(R) : # R is the radius parameter
  graph_list = [] # List of graph datasets

  N = 100000 # number of events

  for n in range(N) :
    particles_list = [] # A list of all the particles in an event

    constituent_indx = [] # A list of indices of constitents of the dijet

    parent_indices = [] # A list of parent indices of constitents of the dijet

    pt_norm = [] # A list of normalized transverse momentum of the constituents of the dijet

    ident = [] # A list containing identity of the particles

    node_ft = [] # Node feature matrix

    edge_ft = [] # Edge attribute matrix

    photon_energy = 0 # Total photon energy
    muon_energy = 0 # Total muon energy
    electron_energy = 0 # Total electron energy
    Ks_energy = 0 # Total reconstructed Ks meson energy
    lambda_energy = 0 # Total reconstructed lambda baryon energy
    lambdabar_energy = 0 # Total reconstructed lambdabar baryon energy

    sts = p_status[n] # Generator status of particles
    p_x = px[n] # X-momentum of particles
    p_y = py[n] # Y-momentum of particles
    p_z = pz[n] # Z-momentum of particles
    E = e[n] # Energy of particles
    pcharge = charge[n] # Charge of particles
    ids = pid[n] # PDG id of particles
    pinds = p_ind[n] # Parent indices of particles
    pbegin = p_begin[n] # Parent begin indices of particles
    pend = p_end[n] # Parent end indices of particles

    for i in range(len(p_x)) :
      part_p4 = PseudoJet(float(p_x[i]), float(p_y[i]), float(p_z[i]), float(E[i]))
      part_p4.set_user_index(i) # setting the index of the particle
      particles_list.append(part_p4)

    stable_p4 = [] # A list of stable particles in the event

    for i in range(len(particles_list)) :
      if sts[i] == 1 :
        stable_p4.append(particles_list[i])

    # Applying clustering of particles with anti-kt algorithm
    cluster = ClusterSequence(stable_p4, JetDefinition(antikt_algorithm, R))
    jet_set = sorted_by_pt(cluster.inclusive_jets(ptmin = 20.0))

    if len(jet_set) > 1 :
      # Dijet kinematics
      dijet = jet_set[0] + jet_set[1]
      d_m = dijet.m() # invariant mass
      d_pt = dijet.pt() # transverse momentum
      d_eta = dijet.eta() # pseudo rapidity
      d_e = dijet.e() # energy

      Dijet_constituents = jet_set[0].constituents() + jet_set[1].constituents() # Dijet constituents

      N_constituents = len(Dijet_constituents) # Total number of constituents N in the dijet

      deta_mean = -0.0013190964951608679 # Mean of dijet eta distributions
      dpt_mean = 37.93450632154195 # Mean of dijet pt distributions
      dN_mean = 25.0419649122807 # Mean of dijet N distributions

      deta_std = 1.1568112511798003 # Standard deviation of dijet eta distributions
      dpt_std = 18.16493460808567 # Standard deviation of dijet pt distributions
      dN_std = 11.041718263218032 # Standard deviation of dijet N distributions

      for i in range(N_constituents) :
        constituent_indx.append(Dijet_constituents[i].user_index())

      constituent_ids = ids[constituent_indx]

      for j in constituent_indx :
        const_id = ids[j]

        if const_id == 22 :
          photon_energy += particles_list[j].e()
        if const_id == 13 :
          muon_energy += particles_list[j].e()
        if const_id == 11 :
          electron_energy += particles_list[j].e()

        parent_indx = get_parent_ids(j, ids, pinds, pbegin, pend)
        parent_indices.append(parent_indx)

        parent_id = ids[parent_indx]

        pt_norm.append(particles_list[j].pt()/d_pt) # Normalized pT

        p_charge = pcharge[j] # Particle charge

        if p_charge > 0 or parent_id == 3122 :
          ident.append(1)
        elif p_charge < 0 or parent_id == 310 :
          ident.append(-1)
        else :
          ident.append(0)

      pd = find_parent_and_daughter_ids(parent_indices, constituent_ids, ids)

      mes_count = 0 # Strange meson count
      bar_count = 0 # Strange baryon count

      for k in range(len(pd)) :
        pd_indx = pd[k][0]
        pd_id = pd[k][1]

        if pd_id == 310 :
          mes_count = 1
          Ks_energy += particles_list[pd_indx].e()
        if pd_id == 3122 :
          bar_count = 1
          lambda_energy += particles_list[pd_indx].e()
        if pd_id == -3122 :
          bar_count = 1
          lambdabar_energy += particles_list[pd_indx].e()

      for l in range(len(constituent_indx)) :
        node_ft.append([(d_eta - deta_mean)/deta_std, (d_pt - dpt_mean)/dpt_std, (N_constituents - dN_mean)/dN_std, photon_energy/d_e, muon_energy/d_e, electron_energy/d_e, Ks_energy/d_e, lambda_energy/d_e, lambdabar_energy/d_e, pt_norm[l], ident[l]])

      x = torch.tensor(node_ft, dtype = torch.float) # Node features matrix

      y = torch.tensor([mes_count, bar_count]) # Target features

      edge_index = torch.tensor(list(itertools.permutations(range(N_constituents), 2))).T # Edge index

      src = edge_index[0].tolist()
      dst = edge_index[1].tolist()

      for i in range(len(src)) :
        node_i = src[i]
        node_j = dst[i]

        edge_ft.append([(d_eta - deta_mean)/deta_std, (d_pt - dpt_mean)/dpt_std, (N_constituents - dN_mean)/dN_std, photon_energy/d_e, muon_energy/d_e, electron_energy/d_e, Ks_energy/d_e, lambda_energy/d_e, lambdabar_energy/d_e, pt_norm[node_i], ident[node_i], pt_norm[node_j], ident[node_j]])

      edge_attr = torch.tensor(edge_ft, dtype = torch.float) # Edge attributes matrix

      graph = Data(x = x, edge_index = edge_index, edge_attr = edge_attr, y = y) # Graph data

      graph_list.append(graph)

  return graph_list

In [14]:
R = get_graph_list(0.8)

In [15]:
from sklearn.model_selection import train_test_split

# Splitting the dataset into training (70%) and temporary (30%)
train_data, temp_data = train_test_split(R, test_size = 0.3, random_state = 42)

# Splitting the temporary dataset into validation (50%) and test (50%)
val_data, test_data = train_test_split(temp_data, test_size = 0.5, random_state = 42)

# DataLoader for each split
train_loader = DataLoader(train_data, batch_size = 64, shuffle = True)
val_loader = DataLoader(val_data, batch_size = 64, shuffle = False)
test_loader = DataLoader(test_data, batch_size = 64, shuffle = False)

print(f"Training dataset size: {len(train_data)}")
print(f"Validation dataset size: {len(val_data)}")
print(f"Test dataset size: {len(test_data)}")


Training dataset size: 69825
Validation dataset size: 14962
Test dataset size: 14963


In [16]:
class MPNN(MessagePassing) :
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr = 'mean')
        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.edge_lin = torch.nn.Linear(13, out_channels)
        self.out_layer = torch.nn.Linear(out_channels, out_channels)

    def forward(self, x, edge_index, edge_attr) :
        x = self.lin(x)
        return self.propagate(edge_index, x = x, edge_attr = edge_attr)

    def message(self, x_j, edge_attr) :
        return x_j + self.edge_lin(edge_attr)

    def update(self, aggr_out) :
        return self.out_layer(aggr_out)

In [17]:
class GraphMPNN(torch.nn.Module) :
    def __init__(self, in_channels, hidden_channels, out_channels = 2) :  # 2 output neurons for classification
        super().__init__()
        self.mpnn = MPNN(in_channels, hidden_channels)
        self.fc = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, data) :
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        x = self.mpnn(x, edge_index, edge_attr)
        x = global_mean_pool(x, batch)
        return self.fc(x)  # No softmax, since BCEWithLogitsLoss expects raw logits

In [18]:
# Define model
model = GraphMPNN(in_channels = 11, hidden_channels = 16, out_channels = 2)

# Optimizer and Binary Cross-Entropy Loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()  # Multi-label classification loss

def evaluate(model, data_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # No gradients needed for evaluation
        for batch in data_loader:
            out = model(batch)
            probs = torch.sigmoid(out)  # Convert logits to probabilities
            preds = (probs > 0.5).int()  # Convert probabilities to binary labels (0 or 1)
            correct += (preds == batch.y.view(-1, 2)).sum().item()
            total += batch.y.numel()  # Count total number of labels (in the batch)
    return correct / total

# Training loop with validation
best_val_acc = 0
for epoch in range(50):
    model.train()  # Set the model to training mode
    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch)
        loss = criterion(out, batch.y.view(-1, 2).float())
        loss.backward()
        optimizer.step()

    # Evaluate on validation set after each epoch
    val_acc = evaluate(model, val_loader)
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}, Validation Accuracy: {val_acc * 100:.2f}%")

    # Save the best model based on validation accuracy
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")

# Load the best model for final testing
model.load_state_dict(torch.load("best_model.pth", weights_only = True))

# Evaluate on the test set
test_acc = evaluate(model, test_loader)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

Epoch 1, Loss: 0.0039, Validation Accuracy: 97.05%
Epoch 2, Loss: 0.0010, Validation Accuracy: 97.27%
Epoch 3, Loss: 0.0085, Validation Accuracy: 99.37%
Epoch 4, Loss: 0.0106, Validation Accuracy: 98.51%
Epoch 5, Loss: 0.0093, Validation Accuracy: 99.35%
Epoch 6, Loss: 0.0722, Validation Accuracy: 97.92%
Epoch 7, Loss: 0.0041, Validation Accuracy: 99.10%
Epoch 8, Loss: 0.0240, Validation Accuracy: 97.51%
Epoch 9, Loss: 0.0039, Validation Accuracy: 99.13%
Epoch 10, Loss: 0.0007, Validation Accuracy: 98.92%
Epoch 11, Loss: 0.0020, Validation Accuracy: 99.18%
Epoch 12, Loss: 0.0002, Validation Accuracy: 99.44%
Epoch 13, Loss: 0.0787, Validation Accuracy: 99.31%
Epoch 14, Loss: 0.0016, Validation Accuracy: 99.41%
Epoch 15, Loss: 0.0145, Validation Accuracy: 99.51%
Epoch 16, Loss: 0.0121, Validation Accuracy: 99.76%
Epoch 17, Loss: 0.0090, Validation Accuracy: 99.42%
Epoch 18, Loss: 0.0000, Validation Accuracy: 99.66%
Epoch 19, Loss: 0.0114, Validation Accuracy: 98.66%
Epoch 20, Loss: 0.000